In [1]:
import sys

In [2]:
sys.path.append('/Users/diegodavila/miniconda3/lib/python3.7/site-packages')

In [3]:
import flywheel
import datetime
import time
import pytz

fw = flywheel.Client()
proj = fw.lookup('bbl/GRMPY_822831')
sessions = proj.sessions()
sessions = [fw.get(x.id) for x in sessions]

/Users/diegodavila/miniconda3/lib/python3.7/site-packages/flywheel/flywheel.py:5758: UserWarning: Client version 9.0.3 does not match server version 11.3.4. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [4]:
def get_latest_fmriprep(session):
    
    if session.analyses:
        
        timezone = pytz.timezone("UTC")
        init_date = datetime.datetime(2018, 1, 1)
        latest_date = timezone.localize(init_date)

        latest_run = None

        for i in session.analyses:
            gear_name = i.gear_info['name']
            state = i.job.state
            date = i.created
            if 'fmriprep' in gear_name and date > latest_date and state =='complete':
                latest_date = date
                latest_run = i
        
        if latest_run is not None:
            fmriprep_out = [x for x in latest_run.files if 'fmriprep' in x.name][0]
            fmriprep_out
            return(fmriprep_out)

        else:
            return None

    else:
        return None

In [5]:
def acquisition_asl2(acq):
    
    asl_to_return = None
    is_asl = False
    #is_multi_volume = False
    
    acq = fw.get(acq.id)
    filelist = [x for x in acq.files if x.type in ["nifti", "dicom"]]
    #print([f.name for f in filelist])
    
    for f in filelist:
        
        if f.type == "nifti":
            #if f.info['BIDS']['Folder'] == "asl":
            if 'BIDS' in f.info:
                if 'Filename' in f.info['BIDS']:
                    if 'asl' in f.info['BIDS']['Filename']:
                        if 'dicomref' in f.name:
                            continue 
                        else:
                            asl_to_return = f
                #print(f.name)
                #print(f.info['BIDS']['Filename'])
                            is_asl = True
        
#         if f.type == "dicom":
#             if "AcquisitionNumber" in f.info:
#                 if f.info['AcquisitionNumber'] >= 80:
#                     #print(f.info['AcquisitionNumber'])
#                     is_multi_volume = True
                    
    #print(is_asl)
    if is_asl:
        return asl_to_return

In [6]:
asl_list = [acquisition_asl2(x) for x in sessions[100].acquisitions()]
asl_list

[None, None, None, {'classification': {'Features': ['Magnitude'],
                     'Intent': ['Functional'],
                     'Measurement': [],
                     'measurement': []},
  'created': datetime.datetime(2019, 2, 26, 21, 51, 2, 964000, tzinfo=tzutc()),
  'hash': 'v0-sha384-31704e87a93aecf56bc5d0361d14a6e1f12e5468ca26aa96cbf39db350c0befeedb2cbd33ab8d66a447d1d16877c31e4',
  'id': '7abd4a79-b05c-4c13-90b3-31a85838e80c',
  'info': {'AcquisitionDateTime': '2018-01-18T15:38:44.545000',
           'AcquisitionMatrixPE': 64,
           'AcquisitionNumber': 1,
           'AcquisitionTime': '15:38:44.545000',
           'BIDS': {'Acq': '',
                    'Ce': '',
                    'Dir': '',
                    'Echo': '',
                    'Filename': 'sub-88402_ses-10679_asl.nii.gz',
                    'Folder': 'asl',
                    'IntendedFor': '',
                    'Mod': '',
                    'Modality': 'bold',
                    'Path': 'sub-88

In [7]:
def get_asl(sess):
    acs = sess.acquisitions()
    asl_list = [acquisition_asl2(x) for x in acs]
    asl_list = [i for i in asl_list if i]
    if len(asl_list) >= 1:
        return asl_list[0]
    else:
        return None

In [8]:
for ses in sessions:
    if get_asl(ses) is not None:
        print(ses.subject.label)

119302
122916
93169
126903
129926
103679
139272
114713
130211
99964
85853
116210
83010
83999
116051
116360
114990
110168
118393
93734
98831
86287
119791
87457
116019
86444
84973
117226
85173
105979
82790
125554
88209
112061
82492
86924
87804
98394
121085
87990
88760
87445
20362
20322
20313
20325
20247
20320
92089
20197
20189
80688
88589
85743
129552
20153
94276
20151
93517
92182
86722
86972
80557
126389
127236
106802
19981
20011
90021
93274
89279
19977
133220
129886
130121
83835
95057
132179
104785
105309
104059
105168
88773
108202
106246
98871
105767
120217
122528
88402
109521
122801
109735
131867
120562
105860
112332
116354
92193
99604
112694
106039
90077
126176
20120
20120
20017
114935
110354
95116
121050
91717
121407
89377
132657
109646
129720
102041
87346
87601
112249
93723
106331
95460
104235
97994
117397
126554
85083
127417
127008
128154
105176
91962
125511
10582
98567
122522
87646
121001
93549
84688
127611
92211
131224
92927
97485
130687
95015
88770
97271
96659
85369
81725
9385

In [10]:
xcp = fw.lookup('gears/xcpengine-fw')

In [11]:
designFile = [x for x in proj.files if x.name == "cbf_new2.dsn"][0]
designFile = proj.get_file(designFile.name)
print(designFile)

{'classification': {},
 'created': datetime.datetime(2020, 5, 5, 0, 17, 41, 994000, tzinfo=tzutc()),
 'hash': 'v0-sha384-aa7301d63248a4038737a3a89aa9967889466312c1b136b280024d0cc2c4e74aa93e77e887a81a5c22aad693871e5b20',
 'id': '1d23ddb5-abd6-4196-9590-e321c5016e93',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/octet-stream',
 'modality': None,
 'modified': datetime.datetime(2020, 5, 5, 0, 17, 40, 14000, tzinfo=tzutc()),
 'name': 'cbf_new2.dsn',
 'origin': {'id': 'diegodav@upenn.edu',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 3307,
 'tags': [],
 'type': None,
 'zip_member_count': None}


In [12]:
xcp_runs = {}

for ses in sessions:

    fmriprep = get_latest_fmriprep(ses)

    asl = get_asl(ses)

    if all([fmriprep, asl]):
        print(ses.label, " has asl")
        myconfig = {
            'analysis_type': 'cbf'
        }

        myinputs = {
            'fmriprepdir': fmriprep,
            'designfile': designFile,
            'img': asl
        }
        xcp_runs[ses.label] = xcp.run(analysis_label="XCP_asl_{}".format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())), destination=ses, inputs=myinputs, config=myconfig)

    else:
        print(ses.label, " No asl")
        xcp_runs[ses.label] = None


11170  No asl
11165  No asl
9942  No asl
10122  has asl
10114  has asl
10113  has asl
10112  has asl
10111  has asl
10110  has asl
10109  has asl
10107  has asl
10106  has asl
10105  has asl
10103  has asl
10102  has asl
10101  has asl
10099  has asl
10098  has asl
10097  has asl
10096  has asl
10095  has asl
10094  has asl
10092  has asl
10084  has asl
10083  has asl
10082  has asl
10081  has asl
10078  has asl
10076  has asl
10075  has asl
10074  has asl
10069  has asl
10067  has asl
10059  has asl
10058  has asl
10057  has asl
10056  has asl
10046  has asl
10035  has asl
10026  has asl
10025  has asl
10015  has asl
9995  has asl
9980  No asl
9970  No asl
9969  No asl
9968  No asl
9958  No asl
9956  No asl
9955  No asl
9947  No asl
11150  has asl
11153  has asl
11149  has asl
11139  has asl
11136  has asl
11128  has asl
11123  has asl
11108  has asl
11104  has asl
10999  has asl
10988  has asl
10982  has asl
10975  has asl
10974  has asl
10976  has asl
10963  has asl
10957  has asl
1

In [13]:
xcp_runs

{'11170': None,
 '11165': None,
 '9942': None,
 '10122': '5ed667106b6c870423adac0b',
 '10114': '5ed667116b6c87041badae01',
 '10113': '5ed667136b6c870427ada9ed',
 '10112': '5ed667146b6c870427ada9f8',
 '10111': '5ed667156b6c870427adaa05',
 '10110': '5ed667166b6c870427adaa0e',
 '10109': '5ed667186b6c870423adac20',
 '10107': '5ed667196b6c870427adaa1e',
 '10106': '5ed6671a6b6c870423adac26',
 '10105': '5ed6671b6b6c870427adaa32',
 '10103': '5ed6671c6b6c870427adaa3b',
 '10102': '5ed6671e6b6c870427adaa46',
 '10101': '5ed6671f6b6c870427adaa51',
 '10099': '5ed667206b6c870427adaa5e',
 '10098': '5ed667216b6c870427adaa68',
 '10097': '5ed667236b6c870427adaa73',
 '10096': '5ed667246b6c870427adaa7b',
 '10095': '5ed667256b6c870427adaa89',
 '10094': '5ed667266b6c870423adac46',
 '10092': '5ed667276b6c870393adaf09',
 '10084': '5ed667296b6c87040badaf13',
 '10083': '5ed6672a6b6c870427adaab2',
 '10082': '5ed6672b6b6c870427adaabc',
 '10081': '5ed6672c6b6c870423adac59',
 '10078': '5ed6672d6b6c870427adaad3',
 '1